In [1]:
import torch
import torch.nn as nn
from train_test import test, train, train_kd, test_batch
from data_loader import get_data_loader,load_checkpoint
from vit import ViT
from vit_CNNFF import ViTCNNFF
from helpers import count_parameters
import time

import torch.nn.utils.prune as prune
import torch.nn.functional as F

In [2]:
torch.manual_seed(42)
train_loader, test_loader = get_data_loader(
    1024, 2, "datasets/cifar-100/cifar-100-python", download=True
)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
model = ViTCNNFF(
    image_size=32,
    patch_size=4,
    num_classes=100,
    dim=64,
    depth=3,
    heads=4,
    mlp_dim=128,
    dropout=0.1,
).to("cuda")
model_base= ViTCNNFF(
    image_size=32,
    patch_size=4,
    num_classes=100,
    dim=64,
    depth=3,
    heads=4,
    mlp_dim=128,
    dropout=0.1,
).to("cuda")


In [4]:
load_checkpoint(model, "save_model/cifar-100/vit16_CNNFF/best_model.pt")

test(model, test_loader, device="cuda")

Test Accuracy: 57.79%


57.79

In [ ]:
for name, module in model.named_modules():
    if isinstance(module, nn.Linear):
        m=prune.ln_structured(module, name="weight", amount=0.5, n=2, dim=0)
        m=prune.remove(m,name="weight")
        print(f"Pruned {name}")

In [ ]:
# layers = list(model.transformer.layers[0][1].net.children())

# linear_layer = layers[0]

# weights = linear_layer.weight
# weights
# model.transformer.layers[0][0].to_qkv.weight

In [ ]:
for layer in model.modules():
    if isinstance(layer, nn.Linear):
        for param_name, param in layer.named_parameters():
            sparse_param = param.cpu().to_sparse()
            sparse_param = sparse_param.to("cuda")
            sparse_param = nn.Parameter(sparse_param)
            setattr(layer, param_name, sparse_param)


In [ ]:
batch_array=[]

for batch in test_loader:
    batch_array.append(batch)   

In [ ]:
inference_time_base=0
start_time = time.time()
for image_batch in batch_array:
    with torch.no_grad():
        test_batch(model, image_batch[0], device="cuda")
inference_time_base += time.time() - start_time
print(f"model_base_time: {inference_time_base} ")



In [ ]:
inference_time_pruned=0
start_time = time.time()
for image_batch in batch_array:
    with torch.no_grad():
        test_batch(model_pruned, image_batch[0], device="cuda")
inference_time_pruned += time.time() - start_time
print(f"model_pruned_time: {inference_time_pruned} ")

In [ ]:
_ = torch.manual_seed (2022)
s = ((torch.randn (3, 5) > 0.0) * torch.randn (3, 5)).to_sparse()   
s

lin = torch.nn.Linear (5, 3)
t = lin (s)
t